inspired by https://github.com/Data-Monkey/GTFS-NetworkX/blob/master/GTFStoGraph.py

GTFS API: https://developers.google.com/transit/gtfs/reference#agencytxt

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

### Import the GTFS Files

In [76]:
ROUTES = pd.read_csv('cta_data/routes.txt')
STOPS = pd.read_csv('cta_data/stops.txt')
TRIPS = pd.read_csv('cta_data/trips.txt')
STOP_TIMES = pd.read_csv('cta_data/stop_times.txt')

we (for now) only want to model subways/metros, which use `route_type=1`, so we get a list of the `route_id`s that correspond ot the subway ("L") lines

In [51]:
subway_route_ids = ROUTES[ROUTES['route_type']==1]['route_id'].values.tolist()
subway_route_ids

['Red', 'P', 'Y', 'Blue', 'Pink', 'G', 'Org', 'Brn']

We only want to track `stations` (as opposed to platforms), which are `location_type=1`

In [67]:
STOPS[STOPS['location_type']==1]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding
11127,40010,NaN,Austin (Blue),NaN,41.870851,-87.776812,1,NaN,0
11128,40020,NaN,Harlem/Lake,NaN,41.886848,-87.803176,1,NaN,1
11129,40030,NaN,Pulaski (Green),NaN,41.885412,-87.725404,1,NaN,1
11130,40040,NaN,Quincy,NaN,41.878723,-87.633740,1,NaN,1
11131,40050,NaN,Davis,NaN,42.047710,-87.683543,1,NaN,1
...,...,...,...,...,...,...,...,...,...
11265,41660,NaN,Lake (Subway),NaN,41.884809,-87.627813,1,NaN,1
11266,41670,NaN,Conservatory-Central Park Drive,NaN,41.884904,-87.716523,1,NaN,1
11267,41680,NaN,Oakton-Skokie,NaN,42.026243,-87.747221,1,NaN,1
11268,41690,NaN,Cermak-McCormick Place,NaN,41.853115,-87.626402,1,NaN,1


In [83]:
list(TRIPS[TRIPS['route_id'].isin(subway_route_ids)]['trip_id'].unique())

[60166141078,
 60166141079,
 60166141080,
 60166141081,
 60166141085,
 60166141086,
 60166141087,
 60166141088,
 60166141089,
 60166141090,
 60166141091,
 60166141092,
 60166141093,
 60166141094,
 60166141095,
 60166141096,
 60166141097,
 60166141098,
 60166141099,
 60166141100,
 60166141101,
 60166141102,
 60166141103,
 60166141104,
 60166141105,
 60166141106,
 60166141107,
 60166141108,
 60166141109,
 60166141110,
 60166141111,
 60166141112,
 60166141113,
 60166141115,
 60166141116,
 60166141117,
 60166141118,
 60166141119,
 60166141120,
 60166141121,
 60166141122,
 60166141123,
 60166141124,
 60166141125,
 60166141126,
 60166141127,
 60166141128,
 60166141129,
 60166141130,
 60166141131,
 60166141132,
 60166141133,
 60166141134,
 60166141136,
 60166141137,
 60166141138,
 60166141139,
 60166141140,
 60166141141,
 60166141142,
 60166141143,
 60166141144,
 60166141145,
 60166141146,
 60166141147,
 60166141148,
 60166141149,
 60166141150,
 60166141151,
 60166141152,
 60166141153,
 60166

In [78]:
STOP_TIMES

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled
0,60161865358,06:31:00,06:31:00,30175,1,Skokie,0,0
1,60161865358,06:37:30,06:37:30,30297,2,Skokie,0,21220
2,60161865358,06:39:00,06:39:00,30026,3,Skokie,0,26031
3,60161865358,06:45:00,06:45:00,30027,4,Howard,0,26031
4,60161865358,06:47:00,06:47:00,30298,5,Howard,0,30842
...,...,...,...,...,...,...,...,...
2781379,6291880309010,19:21:08,19:21:08,11576,2,Bryn Mawr,0,724
2781380,6291880309010,19:22:10,19:22:10,14842,3,Bryn Mawr,0,1567
2781381,6291880309010,19:22:47,19:22:47,11578,4,Bryn Mawr,0,2071
2781382,6291880309010,19:24:00,19:24:00,14844,5,Bryn Mawr,0,2961


### use trip information to determine graph edges

In [5]:
origin_stops = []
dest_stops = []
trip_ids = []

edges_list = []
for trip_id, group in STOP_TIMES.groupby('trip_id'):
    for i in group.index[:-1]:
        origin_stop = STOP_TIMES['stop_id'][i]
        dest_stop = STOP_TIMES['stop_id'][i+1]
        if origin_stop in l_stops['STOP_ID'].values and dest_stop in l_stops['STOP_ID'].values:
            origin_stops.append(origin_stop)
            dest_stops.append(dest_stop)
            trip_ids.append(trip_id)
#             edges_list.append((origin_stop, dest_stop))

KeyboardInterrupt: 

In [61]:
edges = pd.DataFrame({'origin_stop':origin_stops, 'dest_stop':dest_stops, 'trip_id':trip_ids})

In [62]:
edges

,origin_stop,dest_stop,trip_id
0,30175,30297,60161865358
1,30297,30026,60161865358
2,30026,30027,60161865358
3,30027,30298,60161865358
4,30298,30176,60161865358
...,...,...,...
128381,30206,30023,60166205118
128382,30023,30061,60166205118
128383,30061,30220,60166205118
128384,30220,30186,60166205118


In [ ]:
G = nx.Graph()

G.add_nodes_from(edge_list)